In [1]:
!pip3 install lime
!pip3 install imblearn
!pip3 install lightgbm

In [2]:
import sklearn
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.semi_supervised import LabelPropagation
from sklearn import metrics

import numpy as np
from numpy import concatenate

import pandas as pd

from imblearn.over_sampling import SMOTE
from collections import Counter
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder

import lime
import lime.lime_tabular

import lightgbm
from lightgbm import LGBMClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/Colab Notebooks/total_EDA1.csv"

df = pd.read_csv(path)

df_lab = df.drop(df[df['CaseType'].isnull()].index)
df_unlab = df.drop(df[df['CaseType'].isnull() != True].index)

df_y_lab = df_lab['CaseType']
y_lab = list(np.array(df_y_lab))

y_unlab = [-1 for _ in range(len(df_unlab))]


df_X_lab = df_lab.drop('CaseType', axis = 1)
df_X_lab = df_X_lab.drop('entid', axis = 1)
df_X_unlab = df_unlab.drop('CaseType', axis = 1)
df_X_unlab = df_X_unlab.drop('entid', axis = 1)

X_lab = (np.array(df_X_lab)).tolist()
X_unlab = (np.array(df_X_unlab)).tolist()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
X_train = X_lab
y_train = y_lab

In [5]:
y_train = LabelEncoder().fit_transform(y_train)
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [6]:
lightGBM = LGBMClassifier(boosting_type='dart',objective='multiclass')

model = lightGBM

model.fit(X_train, y_train, categorical_feature = 'auto')

LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31,
               objective='multiclass', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [7]:
df_test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/test0.csv")
df_pre = pd.read_csv("/content/drive/My Drive/Colab Notebooks/total_EDA.csv")

df_pre = df_pre.drop(df_pre[df_pre['CaseType'].isnull() != True].index);

test = list(df_test['entid'])

testid = list(df_test['entid'])
entid = list(df_pre['entid'])
labels = [''] * len(entid)

df_pre = df_pre.drop('CaseType', axis = 1)
df_pre = df_pre.drop('entid', axis = 1)

for i in range(len(testid)):
  if testid[i] in entid:
    index = entid.index(testid[i])
    prob = model.predict_proba([df_pre.iloc[index]])[0]
    if prob[0] > 0.5:
      labels[i] = 0
    elif prob[1] > 0.5:
      labels[i] = 1
    elif prob[2] > 0.5:
      labels[i] = 2
    elif prob[3] > 0.5:
      labels[i] = 3
    else:
      labels[i] = 4


from pandas.core.frame import DataFrame
ans_df = DataFrame([testid, labels])
ans_df = ans_df.T
ans_df.to_csv("/content/drive/My Drive/Colab Notebooks/test_ans.csv")
